In [1]:
from data.energy_service import EnergyService

from pandas import DataFrame
from model.series_sample import TimeSeriesSampleAccessor
# Accessor does not work unless working with a specific kernal? TODO: Managing kernals
# https://medium.com/@eleroy/jupyter-notebook-in-a-virtual-environment-virtualenv-8f3c3448247

import json
import os

In [2]:
INFLUX_TS_FMT = "%Y-%m-%dT%H:%M:%SZ"

In [3]:
service = EnergyService()

table, timestamps, lables, data = service.get_readings("daily", "MAC000246", "2011-04-12 10:30:00.0000000", "2012-04-12 10:30:00.0000000")

In [4]:
sample = DataFrame(data=data, columns=lables)
sample['timestamp'] = timestamps

sample.tss.format_index(timestamps, INFLUX_TS_FMT)

In [5]:
sample.describe()

,energy_count,energy_max,energy_mean,energy_median,energy_min,energy_std,energy_sum,mac_id,timestamp
count,132,132,132,132,132,132,132,132,132
unique,2,121,132,99,44,132,132,1,132
top,48,1.034,0.18704166666666674,0.083,0.025,0.2904250116868351,9.118,MAC000246,2011-12-17T00:00:00Z
freq,131,2,1,3,9,1,1,132,1


In [6]:
sample.head()

,energy_count,energy_max,energy_mean,energy_median,energy_min,energy_std,energy_sum,mac_id,timestamp
2011-12-03,30,1.179,0.30023333333333324,0.11499999999999999,0.047,0.361230911280261,9.006999999999998,MAC000246,2011-12-03T00:00:00Z
2011-12-04,48,1.9140000000000001,0.4325625041666667,0.175,0.026000000000000002,0.484151596297421,20.7630002,MAC000246,2011-12-04T00:00:00Z
2011-12-05,48,0.934,0.12541666666666665,0.078,0.025,0.15332677290930027,6.02,MAC000246,2011-12-05T00:00:00Z
2011-12-06,48,1.216,0.27754166875,0.114,0.027000000000000003,0.31339866764908253,13.3220001,MAC000246,2011-12-06T00:00:00Z
2011-12-07,48,0.873,0.1887916666666667,0.10350000000000001,0.026000000000000002,0.20961301306986393,9.062000000000001,MAC000246,2011-12-07T00:00:00Z


In [7]:
stationality = sample.tss.stationality('energy_mean')

Stationality of energy_mean


In [8]:
auto_corr = sample.tss.autocorrelation('energy_mean')